In [1]:
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
from uuid import UUID
from pprint import pprint

In [ ]:
username = "tkinlaw_ent"
password = "Esri.4.GIS"
url = "https://adsrv2019.ad.local/server/rest/services/Hosted/SupplyChain/KnowledgeGraphServer"
gis = GIS(r"https://adsrv2019.ad.local/portal", client_id="5PEin6xagJkBNTnX", verify_cert=False)
knowledge_graph = KnowledgeGraph(url=url, gis=gis)
knowledge_graph

### Connect

In [2]:
username = "tkinlaw_ent"
password = "Esri.4.GIS"
url = "https://adsrv2019.ad.local/server/rest/services/Hosted/SupplyChain/KnowledgeGraphServer"
gis = GIS(r"https://adsrv2019.ad.local/portal", client_id="5PEin6xagJkBNTnX", verify_cert=False)
knowledge_graph = KnowledgeGraph(url=url, gis=gis)
knowledge_graph

Setting `verify_cert` to False is a security risk, use at your own risk.


Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://adsrv2019.ad.local/portal/sharing/rest/oauth2/authorize?response_type=code&client_id=5PEin6xagJkBNTnX&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=ieft5rl8JcaabutnJk4G9Azif6Bc6Q&allow_verification=false


### Get list of part names to process

In [3]:
partNameQuery = """MATCH (p:Product)
WITH DISTINCT p.Parts as parts
RETURN parts"""
x=knowledge_graph.query(partNameQuery)
y = []
partNames = []
for i in range(len(x)):
    y.extend(x[i])
for item in y:
    temp = item.split(", ")
    for s in temp:
        partNames.append(s)
partNames = list(set(partNames))
print(partNames)

['Part 7', 'Part 1', 'Part 2', 'Part 3', 'Part 6', 'Part 5', 'Part 4']


### Query

In [4]:
def submitQuery(partName):
    opQuery = """MATCH (product:Product)
    WHERE any(name in split(product.Parts, ", ") WHERE name = "{0}")
    MATCH (part:Part)
    WHERE part.PartName = "{0}"
    RETURN product, part""".format(partName)
    print(opQuery)

    queryResults = knowledge_graph.query(opQuery)
    return queryResults

### Given returned nodes, create relationships

In [5]:
def createRelationships(queryResults):
    countRels = 0
    for p, d in queryResults:
        print("Related nodes:")
        print(p['_properties']['ProductName'])
        print(d['_properties']['PartName'])
        print(d['_id'])
        add_relationship = {
            "_objectType": "relationship",
            "_typeName": "UsedIn",
            "_originEntityId": UUID(f"{d['_id']}"), # part ID
            "_destinationEntityId": UUID(f"{p['_id']}") # Plant ID
        }
        knowledge_graph.apply_edits(adds=[add_relationship])
        countRels += 1
        #print("EDITS MADE")
    print(f"All done! {countRels} relationships created.")

### Perform the edits

In [6]:
for part in partNames:
    print(f"Finding related Products for {part}")
    queryNodes = submitQuery(part)
    createRelationships(queryNodes) 
print("End of script.")

Finding related Products for Part 7
MATCH (product:Product)
    WHERE any(name in split(product.Parts, ", ") WHERE name = "Part 7")
    MATCH (part:Part)
    WHERE part.PartName = "Part 7"
    RETURN product, part
Related nodes:
Product 1
Part 7
f2cb6ea5-176d-4ae4-a73d-b5c1d1b1afde
Related nodes:
Product 2
Part 7
f2cb6ea5-176d-4ae4-a73d-b5c1d1b1afde
Related nodes:
Product 3
Part 7
f2cb6ea5-176d-4ae4-a73d-b5c1d1b1afde
Related nodes:
Product 4
Part 7
f2cb6ea5-176d-4ae4-a73d-b5c1d1b1afde
Related nodes:
Product 6
Part 7
f2cb6ea5-176d-4ae4-a73d-b5c1d1b1afde
Related nodes:
Product 8
Part 7
f2cb6ea5-176d-4ae4-a73d-b5c1d1b1afde
Related nodes:
Product 11
Part 7
f2cb6ea5-176d-4ae4-a73d-b5c1d1b1afde
Related nodes:
Product 13
Part 7
f2cb6ea5-176d-4ae4-a73d-b5c1d1b1afde
All done! 8 relationships created.
Finding related Products for Part 1
MATCH (product:Product)
    WHERE any(name in split(product.Parts, ", ") WHERE name = "Part 1")
    MATCH (part:Part)
    WHERE part.PartName = "Part 1"
    RETU